# <center><u>German Credit Data Set</u></center>

Note: Class imbalance has been solved using Stratified CV which is mentioned in when we performed CV.

## 1. Importing Libraries

In [ ]:
import numpy as np
import pandas as pd

#visualizations
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

import plotly.offline as py
py.init_notebook_mode(connected = True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

#metrics and split
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score

#models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')


## 2. Loading Data

In [ ]:
df = pd.read_csv('../input/german-credit-data-with-risk/german_credit_data.csv')

## 3. Data Description

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
df.info()

Saving accounts and Checking account attributes have missing values

In [ ]:
df.describe()

## 4. Data Quality Assessment & Statistical Analysis

### 4.1. Filling missing Values

In [ ]:
df.isnull().sum()

#### 4.1.1 Savings account Attribute

In [ ]:
df['Saving accounts'].isnull().sum()

Total number of records is 1000 out of which 183 are null values in the Savings Account attrobute which accounts for 18.3% missing records. We therefore will fill these values using MCT

In [ ]:
#mode of Saving Accounts attribute

mode_sav = df['Saving accounts'].mode()
mode_sav

In [ ]:
df['Saving accounts'].fillna(mode_sav[0], inplace=True)

In [ ]:
df['Saving accounts'].isnull().sum()

#### 4.1.2. Checking account Attribute

In [ ]:
df['Checking account'].isnull().sum()

Too many records are missing and for the better analysis of the data set with total 1000 records it will be appropiate to fill these values rather than dropping them

In [ ]:
#mode of Checking Accouny
mode_check = df['Checking account'].mode()
mode_check

In [ ]:
df['Checking account'].fillna(mode_check[0], inplace=True)

In [ ]:
df['Checking account'].isnull().sum()

In [ ]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.describe()

<b>The given data set has 4 categorical and 4 numeric attributes/features. The target variable is also categorical.
<br>
<br>
Youngest customer in our data set is 19 years old and the oldest is 75 years old. The average age of people in the given data set is 35. 
<br>
<br>
Majority of the customers lie in skilled class.
<br>
<br>
The max Credit amount granted to customer is 18424 while minimum was 250. The average credit amount granted is 3271. </b>

## 5.Univariate Analysis

In [ ]:
#labels
lab = df["Risk"].value_counts().keys().tolist()
#values
val = df["Risk"].value_counts().values.tolist()

trace = go.Pie(labels = lab, values = val, marker = dict(colors = [ 'royalblue','lime'], line = dict(color ="white",width =2)),
               rotation = 100,hoverinfo = "label+value+text", textinfo='label+percent',hole = .5)

layout = go.Layout(dict(title = "Risk Count",paper_bgcolor = "rgb(243,243,243)", plot_bgcolor  = "rgb(243,243,243)"))
                        
data = [trace]

fig = go.Figure(data = data,layout = layout)

py.iplot(fig)

<b>The above plot shows that our data has class imbalance and therefore when using machine learning algorithms we will utilize stratified k fold cross validation that is specifically used to address class imbalance.

In [ ]:
#Defining funtion for ploting categorical attributes

def categorical_plots(var, data):
        
    #Adjustment of plots, bigger size and space b/w subplots
    
    fig = plt.figure(figsize=(15,5))
    fig.subplots_adjust(wspace=0.3)
    
    plt.style.use('ggplot')
        
    #1st Plot:  Bar plot     
        
    plt.subplot(1,2,1)
    sns.countplot(x=var, data= data)
    plt.xticks(rotation = 45, horizontalalignment='right')
    plt.xlabel(var.name + ' Distribution')
    

    #2nd Plot: PIE Chart
    
    labels =var.value_counts().index  #Labels that will be written against slices in pie charts
    
    #For the slice with highest value to be exploded, explode parameter is passed. Using for loop to make a tuple of 
    # number of slice using len(unique) and exploding the first slice by mentioning 0.1 at first index. Atlast converted list to tuple
    
    a=[0.1]
    for i in range ((len(var.unique()))-1):
        a.append(0)

    explode1= tuple(a)
    #if var.name != 'Customer Name':
    ax1 = plt.subplot(1,2,2)
    ax1.pie(var.value_counts(), labels=labels,autopct='%1.1f%%', shadow=True,explode= explode1 )
    ax1.axis('equal')
    plt.xlabel(var.name + ' Distribution')
    plt.legend()
        
    show=plt.show()
    
    return(show)

In [ ]:
#FOR NUMERICL PLOTS WE WILL BE USING THE FOLLOWING FUNCTION

def numerical_plots(var):
    
    #Adjustment of plots, bigger size and space b/w subplots
    
    fig = plt.figure(figsize=(15,4))
    fig.subplots_adjust(wspace=0.3)
    
    #1st Plot:  Histogram with KDE plot          
 
    plt.subplot(1,3,1)
    sns.distplot(var, color='b')
    plt.xlabel(var.name + ' Distribution')
    
    
    #2nd Plot:  Box plot
    
    plt.subplot(1,3,2)
    sns.boxplot(y=var)
    plt.xlabel(var.name + ' Distribution')
    

    #3rd Plot:  Histogram    

    plt.subplot(1,3,3)
    var.hist(bins=15, color='steelblue', edgecolor='black', linewidth=1.0, xlabelsize=8, ylabelsize=8, grid=False)
    plt.xlabel(var.name + ' Distribution')
    #plt.grid(color='white')
    
    
    show=plt.show()
    
    return(show)

#### 5.1.1. Age Attribute

In [ ]:
numerical_plots(df.Age)

The plots show that people aged between 20 to 40 tend to apply for credits more as compared to other age groups.
<br>Among the customers who applied for credit, 50% or less were aged around 35 years.


#### 5.1.2.Sex Attribute

In [ ]:
categorical_plots(df.Sex, df)

The plot shows that males apply more than female. Our data shows that 69% of the people who applied for credit were male, while female were 31%.

#### 5.1.3. Job Attribute

In [ ]:
categorical_plots(df.Job,df)

From 1000 people who applied for loan, highly skilled people were more than 600. That accounts for 63% of the total people who applied for credit.


#### 5.1.4. Housing Attribute

In [ ]:
categorical_plots(df.Housing,df)

Customers who have their own properties/houses are in greater number who apply for loan.

#### 5.1.5. Saving Accounts attribute

In [ ]:
categorical_plots(df['Saving accounts'],df)

The plot shows that people who have little savings apply for loan more often as compared to people who have better or more savings.

#### 5.1.6. Checking account

In [ ]:
categorical_plots(df['Checking account'],df)

People who have little amount in their checking accounts are the one in greater number who applied for loan.

#### 5.1.7. Credit Amount

In [ ]:
numerical_plots(df['Credit amount'])

The distribution shows that aplications for credit amount less than 5000 are more as compared to higher loan applications.
<br> Around 50% application were for credit amount =2500

#### 5.1.8. Duration attribute

In [ ]:
numerical_plots(df.Duration)

The distribution shows that majority of the application are loan are for a duration between 10-25 months.

#### 5.1.9. Purpose Attribute

In [ ]:
categorical_plots(df.Purpose,df)

The plots clearly show that most of the loan applications are for buying cars. That accounts for 33.7% application. Interestingly the second most common purpose for loan is radio/TV.

#### 5.1.10. Risk Attribute

In [ ]:
categorical_plots(df.Risk,df)

Around 70% of the total application were classified as good risk. 

## 6. BiVariate Analysis

In [ ]:
fig = plt.figure(figsize=(15,10))
fig.subplots_adjust(wspace=0.5, hspace=0.5)

c=1
for i in df.columns:
    if df[i].dtype =='O' and i!='Risk':
        plt.subplot(2,3,c)
        sns.countplot(x=df[i], hue=df.Risk, data=df)
        plt.xticks(rotation = 45, horizontalalignment='right')
        c+=1
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,5))
fig.subplots_adjust(wspace=0.3)

c=1
for i in df.columns:
    if df[i].dtype !='O' and i!='Job':
        plt.subplot(1,3,c)
        sns.boxplot(y=df[i], x=df.Risk)
        #plt.xticks(rotation = 45, horizontalalignment='right')
        c+=1
plt.show()
#for i in df.columns:
 #   if df[i].dtype !='O':
  #      sns.boxplot(y=df[i], x=df.Risk)
   #     plt.show()

A credit loan aplication of amount greater than 3000 will have more probability of being a Bad Risk as compared to Good.
<br>When a loan is taken for a duration greater than 25 months then the probability of being a Bad Risk is high as compared to being Good.


In [ ]:
plt.style.use('seaborn')
sns.pairplot(df)

<b>From the plots we can observe:
    <br>With an increase in age(>45) the number of credit applications decreases.
    <br>Similarly, middle aged and old aged customers tend to apply for loan applications for shorter durations compared to adults.
    

In [ ]:
plt.style.use('default')

fig = plt.figure(figsize=(10,4))
fig.subplots_adjust(wspace=0.5)

plt.subplot(1,3,1)
sns.boxplot(x=df.Sex,y=df['Credit amount'], hue=df.Risk, data=df)

#Age2=np.where(df.Age)
Age2 = np.where((df.Age<20),'TeenAger', np.where((df.Age>=20) & (df.Age<=30),'Adult',np.where((df.Age>30) & (df.Age<=50),'Middle-Aged','Old')))
plt.subplot(1,3,2)
sns.boxplot(x=Age2,y=df['Credit amount'], data=df)
plt.xticks(rotation=45, horizontalalignment='right');

plt.subplot(1,3,3)
sns.boxplot(x=df.Purpose,y=df['Credit amount'], hue=df.Risk, data=df)
plt.xticks(rotation=45, horizontalalignment='right');

In [ ]:
plt.style.use('default')

fig = plt.figure(figsize=(10,4))
fig.subplots_adjust(wspace=0.5)

plt.subplot(1,3,1)
sns.boxplot(x=df.Sex,y=df['Age'], hue=df.Risk, data=df)

#Age2=np.where(df.Age)
#Age2 = np.where((df.Age<20),'TeenAger', np.where((df.Age>=20) & (df.Age<=30),'Adult',np.where((df.Age>30) & (df.Age<=50),'Middle-Aged','Old')))
plt.subplot(1,3,2)
sns.boxplot(x=Age2,y=df['Duration'], hue=df.Risk, data=df)
plt.xticks(rotation=45, horizontalalignment='right');

plt.subplot(1,3,3)
sns.boxplot(x=df.Purpose,y=df['Age'], hue=df.Risk, data=df)
plt.xticks(rotation=45, horizontalalignment='right');

## 7. <b>Summary:</b>

People aged between 20 to 40 tend to apply for credits more as compared to other age groups. The average of people who apply for loan is 35 years.

Males apply for credits/loans more than females. 69% of the people who applied for credit were male, while female were 31%.

From 1000 people who applied for loan, highly skilled people were more than 600. That accounts for 63% of the total people who applied for credit.

Customers who have their own properties/houses are in greater number who applied for loan.

Customers with little savings and little amounts in checking accounts apply for loan more often as compared to people who have better or more savings.

Aplications for credit amount less than 5000 are more as compared to higher loan applications.

The max Credit amount granted to customer was 18424 while minimum was 250. The average credit amount granted was 3271. </b>

Majority of the application for loan are for a duration of 10-25 months.

Most of the loan applications are for cars. That accounts for 33.7% application. Interestingly the second most common purpose for loan is radio/TV.

A credit loan aplication of amount greater than 3000 will have more probability of being a Bad Risk as compared to Good.

When a loan is taken for a duration greater than 25 months then the probability of being a Bad Risk is high as compared to being Good.

## 8. Correlation of Variables 

In [ ]:
df.Risk = np.where(df.Risk =='good',1,0)

In [ ]:
from yellowbrick.features import rank1d, rank2d

fig, axes = plt.subplots(1, 2,figsize=(15,5))

#heatmap using seaborn
corr = df.corr()
plt.subplot(1,2,1)
sns.heatmap(corr, annot=True)

#2 dimensional Ranking using yellow brick
df2 =df.drop(columns=['Sex','Housing','Saving accounts','Checking account','Purpose'])
rank2d(df2, ax=axes[1]);

<b> 1. Credit amount and Duration attributes have a strong postive relationship. Greater the credit amount, greater will be the duration.
    
<br>2. Credit amount and Duration show a negative correlation with the target variabe Risk i.e. Larger credit loan applications may have higher probabilty of risk compared to smaller.

<br>3. Similarly larger duration of loan tend towards Bad Risk applications.</b>

## 9. Chi Square Statistics

In [ ]:
import scipy.stats as s

def chi2(data,target,alpha):
    
    for i in df.columns:
    
        if df[i].dtype == 'O' and i != target:
            col = i

            ov = pd.crosstab(data[col], data[target])
            #max_least_income = ov.loc[ov[' <=50K'].idxmax()].name
            #max_highest_income = ov.loc[ov[' >50K'].idxmax()].name
            plt.style.use('ggplot')
            ov.plot(kind='bar', figsize=(5,5), stacked=True)
            plt.xlabel(i.title())
                 
            chi = s.chi2_contingency(ov)
            chi2_s = chi[0]
            p_value = chi[1]
            dof = chi[2]
            critical_value = s.chi2.ppf(q=1-alpha, df=dof)
            
            print('\n\033[1m\033[4m', col.upper(),':\033[0m \n')
            print('Significance Level = ', alpha)
            print('Degree of Freedom = ', dof)
            print('chi2 = ', chi2_s)
            print('Critical Value = ',critical_value)
            print('p-value = ', p_value)

            if chi2_s >=critical_value or p_value <= alpha :
                print('\nWe reject the null hypotheses, there is a relationship between the two variables \n')
            else:
                print('\nThere is no relationship between the two variables and the null hypotheses is retained \n')
            
            plt.show()
            #print('\033[1mThe bar chart shows that', max_least_income,i,'has the highest number of people with <=50k income and',max_highest_income,i,'has the highest number of people having income >50K \n')

In [ ]:
chi2(df, 'Risk', 0.05)

<b>All categorical variables have a relationship with the target variable except 'Purpose' attribute. Among all the related variables saving account has the highest affect on the Risk.</b>

In [ ]:
df.drop(columns = ['Purpose'], inplace=True) #Since there is no relation ship with the target variable

In [ ]:
df.head()

## 10. Outliers Removal 

<b>In the univariate analysis we saw that age, credit amount and Duration attributes not only had outliers but were also positivey skewed. Therefore, before feeding the data to model we will remove reduce the skewness and remove the outliers using logarithimic transformation.

In [ ]:
outliers = np.log(df[['Age', 'Duration','Credit amount']])

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(8,3))
sns.distplot(outliers.Age, ax=ax1)
sns.distplot(outliers.Duration, ax=ax2)
sns.distplot(outliers["Credit amount"], ax=ax3)

plt.tight_layout()

## 11. Feature Encoding & Data Normalization

#### 11.1. Manual Encoding

In [ ]:
df.Sex =  np.where(df.Sex =='male',1,0)

#### 11.2. One Hot Encoding

In [ ]:
df.Dummies_Housing = pd.get_dummies(df.Housing)
df.Dummies_Housing.head()

In [ ]:
df.Dummies_Saving = pd.get_dummies(df['Saving accounts'])
df.Dummies_Saving.head()


In [ ]:
df.Dummies_Checking = pd.get_dummies(df['Checking account'])
df.Dummies_Checking.head()

In [ ]:
df = pd.concat([df,df.Dummies_Housing,df.Dummies_Saving,df.Dummies_Checking], axis=1)
df.drop(columns=['Housing', 'Saving accounts', 'Checking account'], inplace=True)
df.head()

# <center><u>12. Machine Learning</u></center>

### 12.1. Transforming Features and Target Variable into Array

In [ ]:
y = df['Risk']

x = df.drop(columns=['Risk'])
x.head()

#### 12.1.2. Data Normalization

In [ ]:
from sklearn.preprocessing import StandardScaler
zscore = StandardScaler()

for i in x.columns:
    df[i] = zscore.fit_transform(df[[i]])

In [ ]:
x = x.to_dict(orient='records')

from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer()
x = vec.fit_transform(x).toarray()
x

In [ ]:
y =np.asarray(y)

## <u>12.3. Stratified Cross Validation</u>

<b>To cater the problems associated with class imbalance, we will use stratifed cross validation which will split data having equal portions of our target class in every split.</b>

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, random_state=None)

for train_index, val_index in skf.split(x,y): 
    #print("Train:", train_index, "Validation:", val_index) 
    xtrain, xtest = x[train_index], x[val_index] 
    ytrain, ytest = y[train_index], y[val_index]


### 12.4. Defining Functions for evaluation

In [ ]:
# Defining a function to be used for evaluation for all algorithms

from sklearn.metrics import roc_auc_score,roc_curve,scorer, classification_report, confusion_matrix
import plotly.tools as tls

def evaluation(algorithm):
    #Classification Report
    print ("\n \033[1m Classification report : \033[0m\n",classification_report(ytest,algorithm ))

    #Accuracy
    print ("\033[1mAccuracy Score   : \033[0m",accuracy_score(ytest, algorithm))

    #conf_matrix
    conf_matrix = confusion_matrix(ytest,algorithm)


    #roc_auc_score
    model_roc_auc = round(roc_auc_score(ytest, algorithm),3) 
    print ("\033[1mArea under curve : \033[0m",model_roc_auc)
    fpr,tpr,thresholds = roc_curve(ytest,probabilities[:,1])

    # roc curve plot
    trace1 = go.Scatter(x = fpr,y = tpr,
                        name = "Roc : " + str(model_roc_auc),
                        line = dict(color = ('rgb(22, 96, 167)'),width = 2),
                       )
    trace3 = go.Scatter(x = [0,1],y=[0,1],
                        line = dict(color = ('rgb(205, 12, 24)'),width = 2,
                        dash = 'dot'))
    #confusion matrix plot
    trace2 = go.Heatmap(z = conf_matrix ,
                        x = ["Bad Risk ","Good Risk"],
                        y = ["Bad Risk","Good Risk"],
                        colorscale = "Viridis",name = "matrix" )
    #subplots
    fig = tls.make_subplots(rows=1, cols=2, horizontal_spacing = 0.40,subplot_titles=('ROC Curve','Confusion Matrix'))

    fig.append_trace(trace1,1,1)
    fig.append_trace(trace3,1,1)
    fig.append_trace(trace2,1,2)


    fig['layout'].update(showlegend=False, title="Model performance" ,
                         autosize = False,height = 400,width = 800,
                         plot_bgcolor = 'rgba(240,240,240, 0.95)',
                         paper_bgcolor = 'rgba(240,240,240, 0.95)',
                         xaxis = dict(title = "false positive rate",
                                 gridcolor = 'rgb(255, 255, 255)',
                                 domain=[0, 0.6],
                                 ticklen=5,gridwidth=2),
                        yaxis = dict(title = "true positive rate",
                                  gridcolor = 'rgb(255, 255, 255)',
                                  zerolinewidth=1),
                        margin = dict(b = 20))

    py.iplot(fig)


In [ ]:
#Defining a function using Yellow brick library to be used for evaluation

from yellowbrick.classifier import ConfusionMatrix, ClassificationReport, ROCAUC
from yellowbrick.features import FeatureImportances



def visualize(model):

    
    fig, axes = plt.subplots(1, 3,figsize=(15,5))
    fig.subplots_adjust(wspace=0.7)
    
    visualgrid = [
        #FeatureImportances(model,ax=axes[0][0]),
        ROCAUC(model, ax=axes[1],cmap='RdYlBu'),
        ConfusionMatrix(model,cmap='BuPu', ax=axes[2]),
        ClassificationReport(model, cmap='PuBu',ax=axes[0])
        
    ]

    for viz in visualgrid:
        viz.fit(xtrain, ytrain)
        viz.score(xtest, ytest)
        viz.finalize()

    plt.show()

## 12.5.<u> Logistic Regression</u>

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

#Model Training
model_LG =LogisticRegression()
model_LG.fit(xtrain, ytrain);

#Prediction
y_pred_LG = model_LG.predict(xtest)
probabilities = model_LG.predict_proba(xtest)
fpr,tpr,thresholds = roc_curve(ytest,probabilities[:,1])


### Logistic Regression Evaluation Using Plotly

In [ ]:
evaluation(y_pred_LG)

### Logistic Regression Evaluation using YellowBrick

In [ ]:
visualize(model_LG)

## 12.6. <u>KNN Algorithm</u>

### Applying KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

#Model Traning
model_knn = KNeighborsClassifier()
model_knn.fit(xtrain,ytrain);

#Prediction
y_pred_knn = model_knn.predict(xtest)
probabilities = model_knn.predict_proba(xtest)
fpr,tpr,thresholds = roc_curve(ytest,probabilities[:,1])

### KNN Evaluation Using Plotly

In [ ]:
#print ("\n \033[1m Classification report : \033[0m\n",classification_report(ytest, y_pred_knn))
#print ("\033[1mAccuracy Score   : \033[0m",accuracy_score(ytest, y_pred_knn))
evaluation(y_pred_knn)

### KNN Evaluation Using YellowBrick

In [ ]:
visualize(model_knn)

## 12.7. <u>Naive Bayes Classifier</u>

### Applying Naive Bayes Classifier

In [ ]:
from sklearn.naive_bayes import GaussianNB

#Model Training
model_nb = GaussianNB()
model_nb.fit(xtrain, ytrain);

# Model Prediction
y_pred_nb = model_nb.predict(xtest)
probabilities = model_nb.predict_proba(xtest)
fpr,tpr,thresholds = roc_curve(ytest,probabilities[:,1])

### Naive Bayes Evaluation Using Plotly

In [ ]:
evaluation(y_pred_nb)

### Naive Bayes Evaluation using YellowBrick

In [ ]:
visualize(model_nb)

## 12.8. <u>Decision Tree Classifer</u>

### Applying Decision Tree

In [ ]:
from sklearn import tree

# Model Traning
model_DT = tree.DecisionTreeClassifier()
model_DT.fit(xtrain,ytrain)

#Predictions
y_pred_DT = model_DT.predict(xtest)
probabilities = model_DT.predict_proba(xtest)
fpr,tpr,thresholds = roc_curve(ytest,probabilities[:,1])

### Decision Tree Evaluation Using Plotly

In [ ]:
evaluation(y_pred_DT)

### Decision Tree Evaluation Using YellowBrick

In [ ]:
visualize(model_DT)

## 12.9. <u>Random Forest Classifier</u>

In [ ]:
from sklearn.ensemble import RandomForestClassifier
#Model Training
model_rfc = RandomForestClassifier()
model_rfc.fit(xtrain, ytrain);

#Prediction
y_pred_rfc = model_rfc.predict(xtest)
probabilities = model_rfc.predict_proba(xtest)
fpr,tpr,thresholds = roc_curve(ytest,probabilities[:,1])

### Random Forest Classifier Evaluation Using Plotly:

In [ ]:
evaluation(y_pred_rfc)

### Random Forest Classifier Evaluation Using Yellow Brick

In [ ]:
visualize(model_rfc)

## 12.10. <u>XGBoost</u>

In [ ]:
import xgboost as xgb

#Model Training
model_xgb  = xgb.XGBClassifier()
model_xgb.fit(xtrain, ytrain);

#Prediction
y_pred_xgb = model_xgb.predict(xtest)
probabilities = model_xgb.predict_proba(xtest)
fpr,tpr,thresholds = roc_curve(ytest,probabilities[:,1])

### XGBoost Evaluation Using Plotly:

In [ ]:
#print ("\n \033[1m Classification report : \033[0m\n",classification_report(ytest, y_pred_xgb))
#print ("\n \033[1m Accuracy : \033[0m\n",metrics.accuracy_score(ytest, y_pred_xgb))
evaluation(y_pred_xgb)

### XGBoost Evaluation Using YellowBrick

In [ ]:
visualize(model_xgb)

## 12.11. <u>AdaBoost Classifier</u>

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

#Model Training
model_adaboost = AdaBoostClassifier()
model_adaboost.fit(xtrain, ytrain);

#Prediction
y_pred_ada = model_adaboost.predict(xtest)
probabilities = model_adaboost.predict_proba(xtest)
fpr,tpr,thresholds = roc_curve(ytest,probabilities[:,1])


### AdaBoost Evaluation Using Plotly:

In [ ]:
evaluation(y_pred_ada)

### AdaBoost Evaluation Using Yellow Brick:

In [ ]:
visualize(model_adaboost)

## 13. Model Metrics Comparision

In [ ]:
from sklearn.metrics import f1_score

def model_report(model,training_x,testing_x,training_y,testing_y,name) :
    model.fit(training_x,training_y)
    predictions  = model.predict(testing_x)
    accuracy     = accuracy_score(testing_y,predictions)
    recallscore  = recall_score(testing_y,predictions)
    precision    = precision_score(testing_y,predictions)
    f1score      = f1_score(testing_y,predictions) 
    roc_auc      = roc_auc_score(testing_y,predictions)
    
    
    df = pd.DataFrame({"Model"           : [name],
                       "Accuracy_score"  : [accuracy],
                       "Recall_score"    : [recallscore],
                       "Precision"       : [precision],
                       "f1_score"        : [f1score],
                       "roc-auc"         : [roc_auc]
                      })
    return df

model1 = model_report(model_LG,xtrain,xtest,ytrain,ytest,
                      "Logistic Reg. ")


model2 = model_report(model_rfc,xtrain,xtest,ytrain,ytest,
                     "Random Forest Classifier")


model3 = model_report(model_xgb,xtrain,xtest,ytrain,ytest,
                     "XGBoost.")

model4 = model_report(model_knn,xtrain,xtest,ytrain,ytest,
                      "KNN Classifier")

model5 = model_report(model_nb,xtrain,xtest,ytrain,ytest,
                      "Naive Bayes")

model6 = model_report(model_DT,xtrain,xtest,ytrain,ytest,
                      "Decision Tree")

model7 = model_report(model_adaboost,xtrain,xtest,ytrain,ytest,
                      "AdaBoost")


model_performances = pd.concat([model1,model4,model6,model5,model2, model3, model7],axis = 0).reset_index()

model_performances = model_performances.drop(columns = "index",axis =1)

table  = ff.create_table(np.round(model_performances,4))

py.iplot(table)

# <center>14. <u> Hyper Parameter Tunning</u></center>

#### <b>We will be utilizing GridSearch to find best parameters for our model and train our model on the best found hyperparamters. 

<b><br>Then we will plot validation curve of any one paramaters for range of hypermaters for every model. 
    
   <br>And lastly we will plot learning curves for every model to see if more data provided to the model will result in better performance or not.
    

In [ ]:
from sklearn.model_selection import GridSearchCV
from yellowbrick.model_selection import ValidationCurve
from yellowbrick.model_selection import LearningCurve
from sklearn.model_selection import validation_curve

In [ ]:
#We will define three functions: One for validation curve using yellowbrick and other using scikit learn
# Third Function for Learning Curve using yellowbrick
plt.style.use('ggplot')
#Function1 (Validation Curve using Yellow Brick)

def validation(model, meter, range1):
    i=0
    fig, axes = plt.subplots(2, 2,figsize=(15,10))
    fig.subplots_adjust(wspace=0.3, hspace=0.3)
    
    cv = StratifiedKFold(n_splits=5) #To avoid class imbalance problem, using Stratified cross validation

    visualgrid = [
                ValidationCurve(model, param_name=meter,param_range = range1, cv=cv, scoring="accuracy",ax=axes[0][0]),
                
                ValidationCurve(model, param_name= meter, param_range= range1, cv=cv, scoring="precision",ax=axes[0][1]),
             
                ValidationCurve(model, param_name= meter, param_range= range1, cv=cv, scoring="recall",ax=axes[1][0]),
                
                ValidationCurve(model, param_name= meter, param_range= range1, cv=cv, scoring="f1",ax=axes[1][1])    ]
    
    score=['Accuracy','Precision','Recall','f1']
    
    print('The plots scores are in following order\n:')
    
    for viz in visualgrid:
        print('\n',i+1,'.',score[i])
        i+=1
        viz.fit(x, y)
        viz.finalize()
            
    plt.show()


#Function3 (Learning Curve using Yellow Brick)

def learning (model):
    
    i=0
    fig, axes = plt.subplots(2, 2,figsize=(15,10))
    fig.subplots_adjust(wspace=0.3, hspace=0.3)
    
    cv = StratifiedKFold(n_splits=5) #To avoid class imbalance problem, using Stratified cross validation
    sizes = np.linspace(0.3, 1.0, 10)

    visualgrid = [
                LearningCurve(model, cv=cv, scoring="accuracy",train_sizes=sizes,ax=axes[0][0]),
                
                LearningCurve(model, cv=cv, scoring="precision",train_sizes=sizes,ax=axes[0][1]),
             
                LearningCurve(model, cv=cv, scoring="recall",train_sizes=sizes,ax=axes[1][0]),
                
                LearningCurve(model, cv=cv, scoring="f1",train_sizes=sizes,ax=axes[1][1])    ]
    
    score=['Accuracy','Precision','Recall','f1']
    
    print('The plots scores are in following order\n:')
    
    for viz in visualgrid:
        print('\n',i+1,'.',score[i])
        i+=1
        viz.fit(x, y)
        viz.finalize()
            
    plt.show()
    

In [ ]:
plt.style.use('ggplot')
#Function 2 (Validation curve using scikit learn)

def validation_plots(name,algorithm, feature, range3):
    
    param_range = range3
    #np.logspace(-5, 3)

    cv = StratifiedKFold(n_splits=5) #To avoid class imbalance problem, using Stratified cross validation

    train_scores, test_scores = validation_curve(algorithm, x, y, feature, param_range=param_range, cv=cv)

    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    plt.title("Validation Curve with"+name)
    plt.xlabel(feature)
    plt.ylabel("Score")
    plt.ylim(0.0, 1.1)
    #plt.xlim(-10,10)
    
    #scaling on x axis and plotting the mean score
    plt.plot(param_range, train_scores_mean, label="Training score", color="green", lw=2, linestyle='dashdot')
        
    plt.plot(param_range, test_scores_mean, label="Cross-validation score", color="navy", lw=2, linestyle='dashdot')
        
    plt.legend(loc="best")
    
    plt.show()

## 14.1 <u> Logistic Regression</u>

In [ ]:
LG= LogisticRegression(penalty='l2')
hyperparameters = {'dual':[True,False],'max_iter':[100,110,120,130,140],'C':[0.001,1.0,1.5,2.0,2.5,100]}
LG_classifier =  GridSearchCV(LG, hyperparameters, refit=True, cv=5)
LG_classifier.fit(xtrain,ytrain);
#LG.fit(X_train,y_train);

In [ ]:
print("The best fit value is found out to be :" ,LG_classifier.best_params_)

In [ ]:
y_pred_LG2 = LG_classifier.predict(xtest)
#y_pred_LG2 = LG.predict(X_test)

In [ ]:
print ("\n \033[1m Classification report : \033[0m\n",classification_report(ytest, y_pred_LG2))
print ("\033[1mAccuracy Score   : \033[0m",accuracy_score(ytest, y_pred_LG2))

### 14.1.1. Validation Curve for Hyperameter C (Using Scikit Learn):

In [ ]:
validation_plots('Logistic Regression',LogisticRegression(penalty='l2'),'C',np.arange(0.0001,100))

<b>The validation curve for hyperparameter C clearly shows that model is well generalized on a wide rang of values for C i.e. if we select -1 or 100, for both the values Logistic Regression will have a score af around 0.70-0.71.</b>

### 14.1.2. Validation Curves for HyperParameter C (Using YellowBrick):

In [ ]:
validation(LogisticRegression(penalty='l2'),'C',np.logspace(-5, 3))

# Following are plots with scoring in the following order:

# 1. Accuracy  2 . Precision  3. Recall  4. f1


### 14.1.3. Learning Curves for Logistic Regression (Using YellowBricks)

In [ ]:
learning(LogisticRegression(penalty='l2'))

<b>All the 4 plots show that the training and validation score converge with increasing instances. i.e the models performace will be better as more data is provided to the model.</b>

## 14.2. <u>KNN Classifer</u>

In [ ]:
knn = KNeighborsClassifier()
hyperparameters = {'n_neighbors' : list(range(1,40)), 'p':[1,2]}
knn_classifier =  GridSearchCV(knn, hyperparameters, refit=True, cv=5)
knn_classifier.fit(xtrain,ytrain);
#knn.fit(X_train,y_train);

In [ ]:
print("The best fit value is found out to be :" ,knn_classifier.best_params_)

In [ ]:
y_pred_knn2 = knn_classifier.predict(xtest)
#y_pred_knn2 = knn.predict(X_test)

In [ ]:
print ("\n \033[1m Classification report : \033[0m\n",classification_report(ytest, y_pred_knn2))
print ("\033[1mAccuracy Score   : \033[0m",accuracy_score(ytest, y_pred_knn2))

### 14.2.1. Validation Curve for Hyperparameter n_neighbors (Using Scikit Learn):

In [ ]:
#Validation Curve using Scikit Learn
validation_plots('KNN',KNeighborsClassifier(),'n_neighbors',np.arange(1, 40))

<b>The Validation curve seconds the best paramter found out using gridsearch i.e. k =31 as we can see the training score and validation score converge when the number of neighbors are 31 and more.</b>

### 14.2.2. Validation Curve for Hyperparameter n_neighbors(Using YellowBricks):

In [ ]:
validation(KNeighborsClassifier(),'n_neighbors',np.arange(1, 40))

<b> Its clear from all the four plots that models performance gets better or more generalized as the number of neighbors increase.</b>

### 14.2.3. Learning Curves for KNN (Using YellowBricks):

In [ ]:
learning(KNeighborsClassifier())

<b> The learning curves show that the model performance will remain same even if more data is provided to the model.

## 14.3. <u>Decision Tree</u>

In [ ]:
DT = tree.DecisionTreeClassifier()
hyperparameters = {'criterion' :['gini','entropy'], 'max_features':[2,4,6,8],'max_depth':[0.001,1,10, 20, 25, 30, 40, 50]},
DT_classifier =  GridSearchCV(DT, hyperparameters, refit=True, cv=5)
DT_classifier.fit(xtrain,ytrain);

In [ ]:
print("The best fit value is found out to be :" ,DT_classifier.best_params_)

In [ ]:
y_pred_DT2 = DT_classifier.predict(xtest)

In [ ]:
print ("\n \033[1m Classification report : \033[0m\n",classification_report(ytest, y_pred_DT2))
print ("\033[1mAccuracy Score   : \033[0m",accuracy_score(ytest, y_pred_DT2))

### 14.3.1. Validation Curve for HyperParameter max_depth(Using Plotly):

In [ ]:
validation_plots('DT',tree.DecisionTreeClassifier(),'max_depth',np.arange(0.001, 40))

<b> The Validation curve shows that with increase in depth of the tree the model will tend t be overfit. With lesser depths model will generalize well.

### 14.3.2. Validation Curve for HyperParameter max_depth(Using YellowBricks):

In [ ]:
validation(tree.DecisionTreeClassifier(),'max_depth',np.arange(0.001, 10))

<b> All 4 curves show that increasing depth of tree will make the model overfit. The tree will be more generalized when the depth is less than 0.

### 14.3.3. Learning Curve for Decision Tree(Using YellowBricks):

In [ ]:
learning(DT)

<b> All the learning curve show that there will be no positive impact on the performance of the decision tree even if more data is provided.

## 14.4. <u>Naive Bayes</u>

In [ ]:
NB = GaussianNB()
hyperparameters = {'var_smoothing':np.logspace(0,-9, num=100)}

NB_classifier =  GridSearchCV(NB, hyperparameters, refit=True, cv=5)
NB_classifier.fit(xtrain,ytrain);

In [ ]:
print("The best fit value is found out to be :" ,NB_classifier.best_params_)

In [ ]:
y_pred_NB2 = NB_classifier.predict(xtest)

In [ ]:
print ("\n \033[1m Classification report : \033[0m\n",classification_report(ytest, y_pred_NB2))
print ("\033[1mAccuracy Score   : \033[0m",accuracy_score(ytest, y_pred_NB2))

### 14.4.1.Validation Curve for HyperParameter var_smoothing(Using Plotly):

In [ ]:
validation_plots('NB',GaussianNB(),'var_smoothing',np.logspace(0,-100, num=100))


### 14.4.2. Validation Curve for HyperParameter var_smoothing(Using YellowBrick):

In [ ]:
validation(GaussianNB(),'var_smoothing',np.logspace(0,-9, num=100))

### 14.4.3. Learning Curves for Gaussian Naive Bayes (Using YellowBricks):

In [ ]:
learning(GaussianNB())

<b>The learning curves show that with incresing number of instances the model generalizes.

<b> When more data is provided to the model, it will generalize well which is clear from the learning curves as the training and validation scores converge with increasing data

## 14.6. <u>Random Forest </u>

In [ ]:
RF= RandomForestClassifier()
hyperparameters = {'n_estimators' :[1,2,5,10,15,25,30], 'max_features':[2,4,6,8],'max_depth':[0.01,1,10, 20, 25,30, 40, 50]}
RF_classifier =  GridSearchCV(RF, hyperparameters, refit=True, cv=5)
RF_classifier.fit(xtrain,ytrain);

In [ ]:
print("The best fit value is found out to be :" ,RF_classifier.best_params_)

In [ ]:
y_pred_rfc2 = RF_classifier.predict(xtest)

In [ ]:
print ("\n \033[1m Classification report : \033[0m\n",classification_report(ytest, y_pred_rfc2))
print ("\033[1mAccuracy Score   : \033[0m",accuracy_score(ytest, y_pred_rfc2))

### 14.6.1. Validation Curve for HyperParameter max_depth (Using Plotly):

In [ ]:
validation_plots('Random Forest',RandomForestClassifier(),'max_depth',np.arange(0.001, 40))

<b> The Validation curve shows that with increase in depth of the model the model will tend t be overfit. With lesser depths model will generalize well.


### 14.6.2. Validation Curve for HyperParameter max_depth (Using YellowBricks):

In [ ]:
validation(RandomForestClassifier(),'max_depth',np.arange(0.001, 10))

<b> All 4 curves show that increasing depth of random forest will make the model overfit. The model will be more generalized when the depth is less than 0.

### 14.6.3. Learning Curves for Random Forest (Using YellowBrick):

In [ ]:
learning(RandomForestClassifier())

<b> The learning curves show that there will no positive impact on models performance with increase in number of instances

## 14.7. <u>XGBoost</u>

In [ ]:
XGB= xgb.XGBClassifier()
hyperparameters = {"learning_rate": [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ],"max_depth": [ 3, 4, 5, 6, 8, 10, 12, 15], 
                    "gamma":[ 0.0, 0.1, 0.2 , 0.3, 0.4 ]}
                   
XGB_classifier =  GridSearchCV(XGB, hyperparameters, refit=True, cv=5)
XGB_classifier.fit(xtrain,ytrain);

In [ ]:
print("The best fit value is found out to be :" ,XGB_classifier.best_params_)

In [ ]:
y_pred_xgb2 = RF_classifier.predict(xtest)

In [ ]:
print ("\n \033[1m Classification report : \033[0m\n",classification_report(ytest, y_pred_xgb2))
print ("\033[1mAccuracy Score   : \033[0m",accuracy_score(ytest, y_pred_xgb2))

### 14.7.1. Validation Curve for HyperParamter Learning Rate (Using Plotly):

In [ ]:
validation_plots('XGBoost',xgb.XGBClassifier(),'learning_rate', np.arange(0,100))

<b> The validation curve shows that the model us generalized well on a wide range of learning rate values. The performance stablizes when learning rate reaches near 50.

### 14.7.2. Validation Curves for HyperParamter Learning Rate (Using YellowBricks):

In [ ]:
validation(xgb.XGBClassifier(),'learning_rate',np.arange(0,100))

<b> All the validation curve shows that the model us generalized well on a wide range of learning rate values. The performance stablizes when learning rate reaches near 50

### 14.7.3. Learning Curves for XGBoost (Using yellowBricks):

In [ ]:
learning(xgb.XGBClassifier())

<b>The Learning curves show that there will no positive impact on models performance with increasing data.

## 14.8. <u>AdaBoost</u>

In [ ]:
ADA = AdaBoostClassifier()
#ADA2= AdaBoostClassifier(learning_rate=0.25)
hyperparameters = {"learning_rate": [0.0001,0.05, 0.10, 0.15, 0.20, 0.25,0.26,0.27, 0.30 ]}

ADA_classifier =  GridSearchCV(ADA, hyperparameters, refit=True, cv=5)
ADA_classifier.fit(xtrain,ytrain);
#ADA2.fit(xtrain,ytrain)

In [ ]:
print("The best fit value is found out to be :" ,ADA_classifier.best_params_)

In [ ]:
y_pred_ada2 = ADA_classifier.predict(xtest)
#y_pred_ada3 = ADA2.predict(xtest)

In [ ]:
print ("\n \033[1m Classification report : \033[0m\n",classification_report(ytest, y_pred_ada2))
print ("\033[1mAccuracy Score   : \033[0m",accuracy_score(ytest, y_pred_ada2))

### 14.8.1. Validation Curve for Hyperparameter Learning Rate (using Plotly):

In [ ]:
validation_plots('AdaBoost',AdaBoostClassifier(),'learning_rate',np.arange(1,10))

<b>The validation curve shows that although model generalized well for learning rate >=2 but the overall score will be greaty reduced for both training and validation.

### 14.8.2. Validation Curve for Hyperparameter Learning Rate (using YellowBricks):

In [ ]:
validation(AdaBoostClassifier(),'learning_rate',np.arange(1,10))

<b> Validation curve shows that for learning rate =1 the model has some differnce in the training and validation scores which is acceptable as the overall score is good. But as the learning rate increases the models genrelizes at the same time its overall score decreases.

### 14.8.3. Learning Curves for AdaBoost (Using yellowBricks):


In [ ]:
learning(AdaBoostClassifier())

<b>The learning curves show that training and validation score converge with increase in data and therfore model will generalize will with incresing number of instances.

## 15. Model Metrics After Hyper Parameter Tunning

In [ ]:
def model_report(model,training_x,testing_x,training_y,testing_y,name) :
    model.fit(training_x,training_y)
    predictions  = model.predict(testing_x)
    accuracy     = accuracy_score(testing_y,predictions)
    recallscore  = recall_score(testing_y,predictions)
    precision    = precision_score(testing_y,predictions)
    f1score      = f1_score(testing_y,predictions) 
    roc_auc      = roc_auc_score(testing_y,predictions)
    
    
    df = pd.DataFrame({"Model"           : [name],
                       "Accuracy_score"  : [accuracy],
                       "Recall_score"    : [recallscore],
                       "Precision"       : [precision],
                       "f1_score"        : [f1score],
                       "roc_auc"         : [roc_auc]
                       
                      })
    return df

model1 = model_report(LG_classifier,xtrain,xtest,ytrain,ytest,
                      "Logistic Reg. ")


model2 = model_report(RF_classifier,xtrain,xtest,ytrain,ytest,
                     "Random Forest Classifier")


model3 = model_report(XGB_classifier,xtrain,xtest,ytrain,ytest,
                     "XGBoost.")

model4 = model_report(knn_classifier,xtrain,xtest,ytrain,ytest,
                      "KNN Classifier")

model5 = model_report(NB_classifier,xtrain,xtest,ytrain,ytest,
                      "Naive Bayes")

model6 = model_report(DT_classifier,xtrain,xtest,ytrain,ytest,
                      "Decision Tree")

model7 = model_report(ADA_classifier,xtrain,xtest,ytrain,ytest,
                      "AdaBoost")

    
model_performances = pd.concat([model1,model4,model6,model5,model2, model3,model7],axis = 0).reset_index()

model_performances = model_performances.drop(columns = "index",axis =1)

table  = ff.create_table(np.round(model_performances,4))

py.iplot(table)

### Best Model


<b>From the model metrics summary above we can see that Logistic Regression performed well when considering all the scores as compared to other algorithms. It has the highest f1 score and accuracy. Precision, recall and and roc_auc is also comparable with other algorithms. Hence for our case Logistic Regression performed best.

# <center> **--End --**</center>